In [2]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar
# ProgressBar().register()
import pandas as pd
# import torch 
from sentence_transformers import SentenceTransformer 
from datasets import load_dataset

/usr/local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
df = pd.DataFrame(load_dataset('imdb')['train'])

In [9]:
df = df.sample(12000)
df

,text,label
24992,This movie really kicked some ass. I watched i...,1
21790,I probably doubled my knowledge of Iran when I...,1
24665,"At first I didn't didn't like it that much, al...",1
5244,It's interesting how 90% of the high-vote revi...,0
10979,NOTHING in this movie is funny. I thought the ...,0
...,...,...
19556,I read Holes in 5th grade so when I heard they...,1
15672,"Along with ""Brothers & Sisters"", ""Six Degrees""...",1
10742,I watched this video at a friend's house. I'm ...,0
23232,"There has been a political documentary, of rec...",1


In [10]:
data = df['text'].tolist()
data[:2]

["This movie really kicked some ass. I watched it over and over and it never got boring. Angelina Jolie really kicked some ass in the movie, you should see the movie, you won't be disappointed. And another reason you should see the movie is because the guy from The X-Files is in it, David Duchovny.",
 "I probably doubled my knowledge of Iran when I saw Secret Ballot (2001). Now I know about four times as much (I doubt I learned a whole heck of a lot from Not Without My Daughter (1991)).<br /><br />Offside is a splendid budget Iranian comedy about a group a girls (working individually) to attend a decisive soccer match for their country's place in the World Cup. Women are not allowed to attend soccer matches, so the nation's armed forces have been mobilized to save any women who try to enter from themselves. Some (teen?) girls try to crash the party by dressing as boys, but are caught. The movie is mostly set at this holding pen where the girls are detained by soldiers, awaiting some un

In [11]:
embedder = SentenceTransformerEmbeddings(
    model_name='all-MiniLM-L6-v2',
    multi_process=False,
    show_progress=True
)

In [12]:
vector = embedder.embed_documents(data)
df['vector'] = [i for i in vector]

Batches:   0%|          | 0/375 [00:00<?, ?it/s]

In [16]:
vectorstore = FAISS.from_embeddings(
    text_embeddings=df[['text', 'vector']].to_numpy(),
    embedding=embedder, 
    metadatas=df[['text']].to_dict('records')
) 

In [17]:
import os 
os.makedirs('./output/', exist_ok=True)
vectorstore.save_local('./output/all-MiniLM-L6-v2-vectorstore.db') 

In [40]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key=<API_KEYS>
)

retriever = vectorstore.as_retriever(k=10, lambda_mult=0.2)

I0000 00:00:1721808669.435863     521 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721808669.455477     521 check_gcp_environment.cc:61] BIOS data file does not exist or cannot be opened.


In [44]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | rag_prompt_custom | llm
)

In [55]:
print(rag_chain.invoke('titanic').content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The movie *Titanic* is a romance film about a young couple, Jack and Rose, who fall in love aboard the ill-fated RMS Titanic. The film portrays their forbidden love amidst the backdrop of the ship's tragic sinking. While some criticize the film's romantic plot and acting, others praise its visual effects, emotional impact, and historical accuracy. *Titanic* was a huge commercial success and won numerous awards, including eleven Academy Awards. 

